In [ ]:
# flask 'hello world'

from flask import Flask

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])

def main():
    return str('Hello World!!')

if __name__=='__main__':
    app.run()

In [1]:
# Let's try with our model

In [2]:
from flask import Flask, request, render_template
import pickle
import numpy as np
import pandas as pd
import json
import os
import spotipy

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/getsuccess', methods=['POST', 'GET'])
def get_success():
    if request.method=='POST':
        
        #query comes from the home.html webpage, it is a dictionary with key 'result'
        query=request.form['result']
        
        model = pickle.load(open("../model_train/model.pkl","rb"))

        # Spotify credentials
        with open('../settings.env') as f: 
            env_vars = json.loads(f.read())    
        os.environ['SPOTIPY_CLIENT_ID'] = env_vars['SPOTIPY_CLIENT_ID']
        os.environ['SPOTIPY_CLIENT_SECRET'] = env_vars['SPOTIPY_CLIENT_SECRET']    
        from spotipy.oauth2 import SpotifyClientCredentials
        sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

        # Spotify search
        audio_features = sp.audio_features(sp.search(q=query, type='track', market='US')['tracks']['items'][0]['id'])
    
        # Model prediction
        model_predict = model.predict([[audio_features[0]['danceability'], audio_features[0]['energy']]])
        
        # flask requires returning model_predict variable as string type to be put into result.html webpage
        return render_template('result.html', prediction=str(model_predict[0]))
    
if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jun/2020 10:04:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 10:04:55] "POST /getsuccess HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 10:05:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 10:05:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 10:05:09] "POST /getsuccess HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 10:05:16] "GET / HTTP/1.1" 200 -
